# Prep for integration script

In [1]:
# Imports
import os
import glob
import pickle
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [4]:
# Set paths
sample_metadata_path = "/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/metadata/sample_metadata.tsv"
outdir_path = "/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/metadata/2024_06_03"

# make output directory
os.makedirs(outdir_path, exist_ok=True)

# Metadata

In [5]:
df = pd.read_csv(sample_metadata_path, sep="\t")
df.head()

,donor_id,tissue,sex,age,bmi
0,UNOS_AFC2208,pancreatic islets,male,32,32.3
1,UNOS_AFEA331,pancreatic islets,male,45,29.3
2,UNOS_AFEP022,pancreatic islets,male,62,36.1
3,nPOD_6004_CB,whole pancreas,male,33,30.9
4,nPOD_6004_10x,whole pancreas,male,33,30.9


In [7]:
# Only keep tissue of type "pancreatic islets"
df = df[df["tissue"] == "pancreatic islets"]
df

,donor_id,tissue,sex,age,bmi
0,UNOS_AFC2208,pancreatic islets,male,32,32.3
1,UNOS_AFEA331,pancreatic islets,male,45,29.3
2,UNOS_AFEP022,pancreatic islets,male,62,36.1


# For integrating with SnapATAC2

In [8]:
# Get all the directories with h5adment files in them
sample_dir = "/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/results/sample_annotation"
input_h5ad_paths = glob.glob(os.path.join(sample_dir, "*/atac/snapatac2/clustered.h5ad"))
len(input_h5ad_paths)

5

In [100]:
df = sra_df[['tissue', 'sex', 'age', 'bmi', 'donor id']]

In [10]:
# Get dict of sample_id to path
sample_id_to_h5ad_path = {}
for input_h5ad_path in input_h5ad_paths:
    sample_id = input_h5ad_path.split("/")[-4]
    sample_id_to_h5ad_path[sample_id] = input_h5ad_path
sample_id_to_h5ad_path

{'nPOD_6004_CB': '/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/results/sample_annotation/nPOD_6004_CB/atac/snapatac2/clustered.h5ad',
 'UNOS_AFEA331': '/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/results/sample_annotation/UNOS_AFEA331/atac/snapatac2/clustered.h5ad',
 'UNOS_AFEP022': '/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/results/sample_annotation/UNOS_AFEP022/atac/snapatac2/clustered.h5ad',
 'nPOD_6004_10x': '/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/results/sample_annotation/nPOD_6004_10x/atac/snapatac2/clustered.h5ad',
 'UNOS_AFC2208': '/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/results/sample_annotation/UNOS_AFC2208/atac/snapatac2/clustered.h5ad'}

In [11]:
# Add column
df["input_h5ad_path"] = df["donor_id"].map(sample_id_to_h5ad_path)
df["input_h5ad_path"].head()

/cellar/users/aklie/opt/miniconda3/envs/get_data/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    /cellar/users/aklie/data/datasets/Chiou2021_is...
1    /cellar/users/aklie/data/datasets/Chiou2021_is...
2    /cellar/users/aklie/data/datasets/Chiou2021_is...
Name: input_h5ad_path, dtype: object

In [12]:
# Save the output
df[["input_h5ad_path", "donor_id", 'tissue', 'sex', 'age', 'bmi']].to_csv(os.path.join(outdir_path, "snapatac2_integrate.tsv"), sep="\t", index=False, header=False)

# DONE!

---